## Tarea 3
## Jose Juan Bastida Perez


**Importamos las librerias necesarias**

Pasos previos:
* Clonamos el repositorio con el comando git clone
* Creamos el entorno de conda con el comando conda create --name= seguido del nombre del archivo que vamos a usar para definir el entorno y lo activamos
* Una vez esta activado podemos lanzar jupyter-lab y creamos nuestro notebook

In [ ]:
from scipy.io import wavfile
import IPython
import os
import numpy as np




**Ahora tenemos que definir donde guardaremos los audios o de donde los cogeremos**


In [ ]:
cwd = os.getcwd()  #guardamos en cwd la ruta actual
audio_input_path = os.path.join(cwd, os.path.join('audio', 'ejemplos'))  
audio_output_path = os.path.join(cwd, os.path.join('audio', 'output'))

print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

**Ahora vamos a cargar un archivo de audio de la carpeta ejemplos**


In [ ]:

filename = os.path.join(audio_input_path,'breaking_bad.wav')
sample_rate, audio_data = wavfile.read(filename)  #en sample rate se guardara la frecuencia y en audio data los datos de la cancion
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

**Mostramos la frecuencia de muestreo y en audio data tenemos la información de la cancion**





 


**Para reproducirlo usamos la funcion display.Audio donde es necesario pasarle el audio data y la frecuencia de muestreo**

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo.

**A continuacion vamos a mostrar la informacion principal de la onda, la cual la tenemos en audio_data**


In [ ]:
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}') #aqui se enseña el numero de muestras y el numero de canales
print(f'- 1º canal:   {audio_data[:10, 0]}...') #aqui mostramos 5 elementos del primer canal
print(f'- 2º canal:   {audio_data[:10, 1]}...') #aqui mostramos 5 elementos del primer canal
print(f'- Resolucion: {type(audio_data[0,0])}\n') #mostramos el tipo de dato

**Para pasar un audio a mono, lo que tendriamos que hacer seria juntar los sonidos en un mismo canal, por lo tanto calculamos la media de ambos canales**


In [ ]:

# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Se hace la media de dos muestras, de ambos canales
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')  #aqui se enseña el numero de muestras y el numero de canales (en este caso solo hay 1)
print(f'- Canal unico:  {new_data_mono[:5]}...') #muestra 5 elementos del canal unico

# Mantenemos la misma resolucion que antes y la mostramos para comprobar que este correcta.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')

**Ahora procedemos a guardar el archivo en la ruta que nos indica audio_output_path**

In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'ejemplo_mono.wav'),
    rate=sample_rate,
    data=new_data_mono
)

**Volvemos a escucharlo**

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

**Grafica**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Cargar datos de audio estéreo y mono (asumiendo que ya has cargado los datos)
# audio_data_stereo y audio_data_mono son los arrays NumPy que contienen los datos de audio

# Configuración del tamaño de la figura
plt.figure(figsize=(12, 6))

# Graficar audio estéreo
plt.subplot(2, 1, 1)  # Configurar subplot para el audio estéreo
plt.plot(np.arange(len(audio_data)) / sample_rate, audio_data[:, 0], label='Canal izquierdo')
plt.plot(np.arange(len(audio_data)) / sample_rate, audio_data[:, 1], label='Canal derecho')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Audio Estéreo')
plt.legend()

# Graficar audio mono
plt.subplot(2, 1, 2)  # Configurar subplot para el audio mono
plt.plot(np.arange(len(new_data_mono)) / sample_rate, new_data_mono, color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Audio Mono')

# Ajustar diseño y mostrar la gráfica
plt.tight_layout()
plt.show()


**La principal diferencia entre el sonido es que en estereo se reparte entre los dos canales de sonido, mientras que el sonido en mono solo va por un unico canal.**

## Tarea 4


**Primero añadimos las librerias necesarias**

In [ ]:
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt


*  **Frecuencia de muestreo** : Es el numero de muestras o de veces que tomamos informacion por segundo, por lo que tambien seria, la velocidad con lo que lo hacemos. Cuanto mayor es la frecuencia de muestreo podriamos decir que mayor es la calidad y la resolucion de esta(aunque esto haga que el archivo aumente su peso consideradamente).
* **Aliasing**: Es el efecto sobre las señales continuas que hace que por una baja frecuencia de muestreo, no podamos distinguirla bien respecto a otras cuando son muestreadas digitalmente. Esto se produce por tomas un numero de muestras tan bajo que no tengamos una suficiente informacion acerca de dicha onda y su representacion no da la suficiente informacion.
* **Profundidad de bits**: Nos dice cuantos bits tenemos disponibles para medir la onda.
* **Ancho de banda**: Es la suma de profundidad de bits + la frecuencia de muestreo. Este seria el rango de frecuencias en las que se trabaja, de la mayor a la menor.
* **Tasa de bits**: Nos indica el numero de bits por segundo que deberiamos procesar para que este se escuche de la forma prevista, esto lo hace multiplicando la frecuencia de muestreo, la profundidad de bits y el numero de canales.



**Fournier**

**Primero cargamos un audio mono**

In [ ]:
audio_input_path = os.path.join(cwd, os.path.join('audio', 'output'))  
frecuencia, audio_mono_data = wavfile.read(filename=os.path.join(audio_input_path, 'ejemplo_mono.wav'))
print(f'{audio_mono_data.shape}\n') 




**Ahora vamos a aplicar la transformada de fourier, es decir, descomponer la señal en vez de en el dominio del tiempo en el de la frecuencia**

In [ ]:

n = len(audio_mono_data)  #tamaño del array de los datos
Fs = frecuencia   #frecuencia de muestreo

# Working with stereo audio, there are two channels in the audio data.
# Let's retrieve each channel seperately:
# ch1 = np.array([data[i][0] for i in range(n)]) #channel 1
# ch2 = np.array([data[i][1] for i in range(n)]) #channel 2
# We can then perform a Fourier analysis on the first
# channel to see what the spectrum looks like.

# Calculando la Transformada Rapida de Fourier (FFT) en audio mono (si fuese en estereo tendriamos dos canales con los que trabajar como se ve arriba).
ch_Fourier = np.fft.fft(audio_mono_data) 

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2]) #guardamos en esta variable el valor absoluto de la transformada de fourier, pero se toma solo la mitad hasta n//2 , es decir, solo considera las fracuencias positivas

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier) #Traza un grafico de la amplitud de las frecuencias frente a las frecuencias correspondientes
plt.ylabel('Amplitud', labelpad=10) #etiqueta
plt.xlabel('$f$ (Hz)', labelpad=10) #etiqueta
plt.show() #muestra

**Ahora vamos a calcular la energia del espectograma y la frecuencia de corte**

In [ ]:

eps = [1e-5, .02, .041, .063, .086, .101, .123] #Definimos una serie de valores para epsilon, que es el valor que determina la energia que se conservara al aplicar el umbral

eps = eps[3] #aqui marcamos el que queramos usar
print(f'Epsilon: {eps}')

thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier) #Guardamos el valor del umbral que se utilizara para determinar que frecuencias se conservaran
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

spec_energy = np.cumsum(abs_ch_Fourier) #Aqui se calcula la energía acumulada del espectro,es decir, se suma la energía de cada frecuencia en el espectro para cada punto de frecuencia.

frequencies_to_remove = thr_spec_energy < spec_energy #Aqui estamos creando una mascara que nos servira para determinar las frecuencias a eliminar del espectro.
print(f'Mascara: {frequencies_to_remove}')

f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2) #Es la frecuencia mas alta en el espectro que se conservara después de aplicar el umbral, es decir, el umbral.
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

**Aqui podemos ver la salida eligiendo el de la posicion [3] y que el valor de epsilon seria de 0.063. La frecuencia de corte seria 9643 Hz, que vemos que quedaria sobre la mitad de la grafica**

## Compresion del archivo ##

**Para comprimir el archivo vamos a quedarnos con menos valores del audio original e intentando perder la menor informacion importante necesaria. Los elementos que vamos a seleccionar serias aquellos que esten en posicion multiplo del valor D(slicing). D=fs/f0**

In [ ]:
audio_input_path = os.path.join(cwd, os.path.join('audio', 'output'))  
frecuencia, audio_mono_data = wavfile.read(filename=os.path.join(audio_input_path, 'ejemplo_mono.wav'))
print(f'{audio_mono_data.shape}\n') 

In [ ]:
wav_compressed_file = "audio_mono_comp.wav" # Indicamos el nombre del archivo y lo almacenamos en una variable

D = int(Fs / f0) #obtenemos el valor D
print(f'Factor de downsampling: {D}') #muestra

# #Se utiliza la tecnica de slicing creando nuevos conjuntos de datos con parte de los datos del original (los que estan en las posiciones que estan marcadas por el valor D)
new_data = audio_mono_data[::D] 

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)


# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

## Espectrograma ##

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

#Se calcula y se traza es espectrograma con el primer valor, ax[0], es decor en el primer subplot
Pxx, freqs, bins, im = ax[0].specgram(audio_mono_data, NFFT=1024, Fs=frecuencia, noverlap=512) 
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

#Se calcula y se traza es espectrograma con el segundo valor, ax[1], es decir en el segundo subplot
Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

**Comprobamos los tamaños de ambos archivos**

In [ ]:
!ls -sh /home/usuario/Descargas/P2SM/audio/output/ejemplo_mono.wav
!ls -sh /home/usuario/P2SM/audio/output/ejemplo_mono_comp.wav

**Por ultimo escuchamos ambos audios**

In [ ]:
IPython.display.Audio(audio_mono_data, rate=frecuencia)


In [ ]:
IPython.display.Audio(new_audio_data, rate=Fs)